In [ ]:
!nvidia-smi

# 匯入函式庫

In [ ]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


# CPU/GPU、自定義資料集、模型、訓練函數

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def SetSeed(myseed):
    # Python random module
    random.seed(myseed)
    # Numpy
    np.random.seed(myseed)
    # Torch
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(myseed)
        torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, WindowSize):
        X = np.expand_dims(X, 1)
        self.X = torch.from_numpy(X)
        self.WindowSize = WindowSize
        
    def __len__(self):
        return len(self.X) - self.WindowSize

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.WindowSize], self.X[idx+self.WindowSize])
        # return (X = [seqs, features], y)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        
        self.Input_HiddenLayer = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=num_layers)

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        hidden, _ = self.Input_HiddenLayer(input)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [ ]:
class Self_Attention(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        self.num_layers = num_layers
        
        self.Input_First_HiddenLayer = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1)
        self.Query_Key_Value_1 = nn.ModuleList([nn.Linear(1, hidden_size), nn.Linear(1, hidden_size), nn.Linear(1, hidden_size)])

        self.Second_And_Following_HiddenLayer = nn.ModuleList([nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1) for i in range(num_layers - 1)])
        self.List_of_Query_Key_Value = [nn.ModuleList([nn.Linear(hidden_size, hidden_size) for q_k_v in range(3)]) for i in range(num_layers - 1)]

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        Query1 = self.Query_Key_Value_1[0](input)
        Key1 = self.Query_Key_Value_1[1](input)
        Value1 = self.Query_Key_Value_1[2](input)
        hidden, _ = self.Input_First_HiddenLayer(Query1, Key1, Value1)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        if self.num_layers > 1:
            for i, Second_And_Following_HiddenLayer in enumerate(self.Second_And_Following_HiddenLayer):
                Query = self.List_of_Query_Key_Value[i][0](hidden)
                Key = self.List_of_Query_Key_Value[i][1](hidden)
                Value = self.List_of_Query_Key_Value[i][2](hidden)
                hidden, _ = Second_And_Following_HiddenLayer(Query, Key, Value)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [ ]:
def train_under_config(model_type,
                       forex_data,
                       length_input_sequence,
                       num_epochs,
                       num_hidden_layers,
                       num_hidden_sizes,
                       batch_sizes,
                       device):
    '''
    model_type: LSTM or Self_Attention,
    forex_data,
    length_input_sequence,
    num_epochs,
    learning_rate,
    num_hidden_layers,
    num_hidden_sizes,
    batch_sizes,
    device
    '''
    # setseed
    SetSeed(9527)
    # dataset
    training_data = forex_data.loc['1981-01-01':'2016-12-31'] # training_data = training set + validation set
    training_dataset = TimeSeriesDataset(training_data, length_input_sequence)
    # dataloader
    training_dataloader = DataLoader(training_dataset, batch_size=batch_sizes, shuffle=True)
    # model
    model = model_type(num_hidden_layers, num_hidden_sizes).double()
    # criterion & optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    # training
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for X, y in training_dataloader:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            ypred = model(X)
            loss = criterion(ypred, y)
            loss.backward()
            optimizer.step()
    
    return model

In [ ]:
# residue pred
def ypred(data, length_input_sequence, model, device):
    '''
    data,
    length_input_sequence,
    model,
    device
    '''
    # dataset
    testing_start_index = len(data.loc['1981-01-01':'2016-12-31']) - length_input_sequence
    testing_end_index = len(data.loc['1981-01-01':'2020-12-31'])
    testing_data = data[testing_start_index:testing_end_index]
    testing_dataset = TimeSeriesDataset(testing_data, length_input_sequence)
    # dataloader
    data_len = len(testing_dataset)
    testing_dataloader = DataLoader(testing_dataset, batch_size=data_len, shuffle=False)
    # evaluating
    model.to(device)
    model.eval()
    for X, y in  testing_dataloader:
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            ypred = model(X)
            ypred = np.array(ypred.cpu())
    return ypred

In [ ]:
def prediction_on_testset(model_type, linear_data, residue_data, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs):
    model = train_under_config(model_type,
                               residue_data,
                               length_input_sequence,
                               num_epochs,
                               num_hidden_layers,
                               num_hidden_sizes,
                               batch_sizes,
                               device)
    model_type_residue_pred = ypred(residue_data, length_input_sequence, model, device)
    model_type_pred = linear_data + model_type_residue_pred
    return model_type_pred

In [ ]:
# random walk prediction
def rw_pred(testing_data, length_input_sequence):
    return np.zeros(len(testing_data[length_input_sequence:]))

In [ ]:
def root_mean_squared_error(ypred, ytrue):
    squared_error = (ypred - ytrue) ** 2
    mean_squared_error = sum(squared_error) / len(squared_error)
    return np.sqrt(mean_squared_error)

In [ ]:
def mean_absolute_error(ypred, ytrue):
    absolute_error = abs(ypred - ytrue)
    return sum(absolute_error) / len(absolute_error)

In [ ]:
def direction_accuracy(ypred, ytrue):
    ypred_elementwise_dot_ytrue = ypred * ytrue
    same_direction_rate = sum(ypred_elementwise_dot_ytrue > 0) / len(ypred_elementwise_dot_ytrue)
    return same_direction_rate

# 匯入訓練資料（訓練加驗證集）訓練，產生預測資料並計算模型的 RMSE, MAE, DA

## Recursive window

In [ ]:
recursive_linear = pd.read_csv('./recursive_linear_prediction.csv', index_col=0)
recursive_linear = recursive_linear.astype('float64')
recursive_linear.index = pd.to_datetime(recursive_linear.index)

cad_recursive_linear = recursive_linear.iloc[:,0]
aud_recursive_linear = recursive_linear.iloc[:,1]
gbp_recursive_linear = recursive_linear.iloc[:,2]

recursive_linear

In [ ]:
recursive_residue = pd.read_csv('./recursive_linear_residue.csv', index_col=0)
recursive_residue = recursive_residue.astype('float64')
recursive_residue.index = pd.to_datetime(recursive_residue.index)

cad_recursive_residue = recursive_residue.iloc[:,0]
aud_recursive_residue = recursive_residue.iloc[:,1]
gbp_recursive_residue = recursive_residue.iloc[:,2]

recursive_residue

### LSTM

In [ ]:
model_type = LSTM

#### cad

In [ ]:
cad_recursive_linear

In [ ]:
cad_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
cad_recursive_LSTM_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_recursive_linear

In [ ]:
aud_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
aud_recursive_LSTM_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_recursive_linear

In [ ]:
gbp_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
gbp_recursive_LSTM_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [ ]:
model_type = Self_Attention

#### cad

In [ ]:
cad_recursive_linear

In [ ]:
cad_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
cad_recursive_Self_Attention_pred = prediction_on_testset(model_type, cad_recursive_linear, cad_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_recursive_linear

In [ ]:
aud_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
aud_recursive_Self_Attention_pred = prediction_on_testset(model_type, aud_recursive_linear, aud_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_recursive_linear

In [ ]:
gbp_recursive_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
gbp_recursive_Self_Attention_pred = prediction_on_testset(model_type, gbp_recursive_linear, gbp_recursive_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

## Rolling window

In [ ]:
rolling_linear = pd.read_csv('./rolling_linear_prediction.csv', index_col=0)
rolling_linear = rolling_linear.astype('float64')
rolling_linear.index = pd.to_datetime(rolling_linear.index)

cad_rolling_linear = rolling_linear.iloc[:,0]
aud_rolling_linear = rolling_linear.iloc[:,1]
gbp_rolling_linear = rolling_linear.iloc[:,2]

rolling_linear

In [ ]:
rolling_residue = pd.read_csv('./rolling_linear_residue.csv', index_col=0)
rolling_residue = rolling_residue.astype('float64')
rolling_residue.index = pd.to_datetime(rolling_residue.index)

cad_rolling_residue = rolling_residue.iloc[:,0]
aud_rolling_residue = rolling_residue.iloc[:,1]
gbp_rolling_residue = rolling_residue.iloc[:,2]

rolling_residue

### LSTM

In [ ]:
model_type = LSTM

#### cad

In [ ]:
cad_rolling_linear

In [ ]:
cad_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
cad_rolling_LSTM_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_rolling_linear

In [ ]:
aud_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
aud_rolling_LSTM_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_rolling_linear

In [ ]:
gbp_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
gbp_rolling_LSTM_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

### Self-Attention

In [ ]:
model_type = Self_Attention

#### cad

In [ ]:
cad_rolling_linear

In [ ]:
cad_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
cad_rolling_Self_Attention_pred = prediction_on_testset(model_type, cad_rolling_linear, cad_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### aud

In [ ]:
aud_rolling_linear

In [ ]:
aud_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
aud_rolling_Self_Attention_pred = prediction_on_testset(model_type, aud_rolling_linear, aud_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

#### gbp

In [ ]:
gbp_rolling_linear

In [ ]:
gbp_rolling_residue

In [ ]:
# optim hyperparameters obtained by grid search
length_input_sequence = 5
num_hidden_layers = 2
num_hidden_sizes = 1600
batch_sizes = 16
num_epochs = 5

In [ ]:
gbp_rolling_Self_Attention_pred = prediction_on_testset(model_type, gbp_rolling_linear, gbp_rolling_residue, length_input_sequence, num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

# 輸出模型預測與比較

## 輸出模型預測

In [ ]:
LSTM_recursive_prediction = pd.concat([cad_recursive_LSTM_pred, aud_recursive_LSTM_pred, gbp_recursive_LSTM_pred], axis=1)
LSTM_recursive_prediction.columns = ['cad_recursive_LSTM_pred', 'aud_recursive_LSTM_pred', 'gbp_recursive_LSTM_pred']
LSTM_recursive_prediction.to_csv('./LSTM_recursive_prediction.csv')
LSTM_recursive_prediction

In [ ]:
LSTM_rolling_prediction = pd.concat([cad_rolling_LSTM_pred, aud_rolling_LSTM_pred, gbp_rolling_LSTM_pred], axis=1)
LSTM_rolling_prediction.columns = ['cad_rolling_LSTM_pred', 'aud_rolling_LSTM_pred', 'gbp_rolling_LSTM_pred']
LSTM_rolling_prediction.to_csv('./LSTM_rolling_prediction.csv')
LSTM_rolling_prediction

In [ ]:
Self_Attention_recursive_prediction = pd.concat([cad_recursive_Self_Attention_pred, aud_recursive_Self_Attention_pred, gbp_recursive_Self_Attention_pred], axis=1)
Self_Attention_recursive_prediction.columns = ['cad_recursive_Self_Attention_pred', 'aud_recursive_Self_Attention_pred', 'gbp_recursive_Self_Attention_pred']
Self_Attention_recursive_prediction.to_csv('./Self_Attention_recursive_prediction.csv')
Self_Attention_recursive_prediction

In [ ]:
Self_Attention_rolling_prediction = pd.concat([cad_rolling_Self_Attention_pred, aud_rolling_Self_Attention_pred, gbp_rolling_Self_Attention_pred], axis=1)
Self_Attention_rolling_prediction.columns = ['cad_rolling_Self_Attention_pred', 'aud_rolling_Self_Attention_pred', 'gbp_rolling_Self_Attention_pred']
Self_Attention_rolling_prediction.to_csv('./Self_Attention_rolling_prediction.csv')
Self_Attention_rolling_prediction

## 載入實際匯率變動率（取對數後一階差分）

## 模型比較